In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plays = pd.read_csv('nfl_data/plays.csv')

pd.set_option('max_colwidth', 100)

In [8]:
# plays.info()

In [9]:
plays[['quarter', 'down', 'yardsToGo','yardlineNumber','preSnapHomeScore', 
    'preSnapVisitorScore', 'absoluteYardlineNumber', 'yardsGained', 'isDropback',
      'playAction']] = plays[['quarter', 'down', 'yardsToGo', 'yardlineNumber','preSnapHomeScore', 
                              'preSnapVisitorScore', 
                              'absoluteYardlineNumber', 'yardsGained', 'isDropback',
                              'playAction'
                              ]].astype(int)



plays[['passLength', 'targetX', 'targetY', 'dropbackDistance',
       'timeToThrow', 'timeInTackleBox', 'timeToSack', 
       'penaltyYards', 'prePenaltyYardsGained']] = plays[['passLength', 'targetX', 'targetY',  'dropbackDistance',
                                                           'timeToThrow', 'timeInTackleBox', 'timeToSack', 
                                                            'penaltyYards', 'prePenaltyYardsGained']].astype('float')

In [10]:
plays['playNullifiedByPenalty'].unique()  # Можно удалить

array(['N'], dtype=object)

In [11]:
plays = plays.drop(['preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability', 'expectedPoints', 
               'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded',
                    'expectedPointsAdded', 'playNullifiedByPenalty'], axis=1)


In [12]:
plays['timeToThrow'] = pd.to_numeric(plays['timeToThrow'], errors='coerce')
plays['timeInTackleBox'] = pd.to_numeric(plays['timeInTackleBox'], errors='coerce')

# Dates

# Basic Info ✅

✅ `gameClock`

`playClockAtSnap`

In [13]:
plays['playClockAtSnap'].head(10)

0    10.0
1     9.0
2    12.0
3    11.0
4     8.0
5    15.0
6    18.0
7     2.0
8     3.0
9    12.0
Name: playClockAtSnap, dtype: float64

In [14]:
plays[['quarter', 'down', 'yardsToGo','yardlineNumber']].describe()

,quarter,down,yardsToGo,yardlineNumber
count,16124.000000,16124.000000,16124.000000,16124.000000
mean,2.572253,1.800794,8.461300,29.226185
std,1.135404,0.827194,3.908658,12.662828
min,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,6.000000,21.000000
50%,3.000000,2.000000,10.000000,30.000000
75%,4.000000,2.000000,10.000000,40.000000
max,5.000000,4.000000,38.000000,50.000000


quarter 5

In [15]:
plays.query('quarter==5 and down==1 and preSnapHomeScore != preSnapVisitorScore')[['gameId', 'playId', 'preSnapHomeScore', 'preSnapVisitorScore']]


,gameId,playId,preSnapHomeScore,preSnapVisitorScore
3735,2022110610,4581,20,17
7675,2022100600,4661,9,12
7977,2022100600,4608,9,12
14615,2022100600,4637,9,12


In [16]:
plays.query('gameId==2022110610').sort_values(by='playId')[['playId', 'quarter', 'down', 'preSnapHomeScore', 'preSnapVisitorScore']]


,playId,quarter,down,preSnapHomeScore,preSnapVisitorScore
4010,64,1,1,0,0
13610,88,1,2,0,0
4118,112,1,1,0,0
8312,141,1,2,0,0
8713,170,1,1,0,0
...,...,...,...,...,...
13574,4525,5,3,17,17
3735,4581,5,1,20,17
14476,4602,5,2,20,17
2197,4621,5,3,20,17


📌 Since each team has one down per quarter, sorting by plays, quarters, and downs is not suitable. We sort by game time instead.

In [17]:
plays.query('gameId==2022110610').query('quarter==5').sort_values(by='gameClock', ascending=False)[['gameClock', 'playId', 'quarter', 'down', 
                                                                                   'preSnapHomeScore', 'preSnapVisitorScore',
                                                                                  'possessionTeam', 'defensiveTeam']]

,gameClock,playId,quarter,down,preSnapHomeScore,preSnapVisitorScore,possessionTeam,defensiveTeam
14898,09:47,4255,5,1,17,17,KC,TEN
15120,09:08,4279,5,2,17,17,KC,TEN
1139,08:27,4303,5,1,17,17,KC,TEN
8380,07:49,4324,5,2,17,17,KC,TEN
6273,07:04,4348,5,3,17,17,KC,TEN
5834,06:32,4383,5,1,17,17,KC,TEN
1363,05:53,4407,5,2,17,17,KC,TEN
15136,05:17,4428,5,3,17,17,KC,TEN
7777,05:13,4450,5,4,17,17,KC,TEN
11371,04:27,4481,5,1,17,17,KC,TEN


An increase in preSnapHomeScore from 17 to 20 indicates that the home team (KC) scored 3 points. This is most likely the result of a successfully executed field goal.

In the initial plays where Kansas City had possession, the team was simply gaining 10 yards every two plays and steadily advancing down the field.

✅ ok

## `yardsGained` ✅

In [18]:
# plays['yardsGained'].hist(bins=100)
# plt.xlim([-25, 25])

In [19]:
# plays['yardsGained'].describe()

In [20]:
# plays.loc[9499]

If Buffalo is attacking from their 2-yard line, how can their absoluteYardlineNumber be 108 instead of 12?

Should we replace absoluteYardlineNumber with 12?

# Where the play started

## `absoluteYardlineNumber` , `yardlineNumber`

In [21]:
# plays['absoluteYardlineNumber'].describe()

In [22]:
# plays.query('absoluteYardlineNumber==109')['yardlineNumber'].unique() # ok

In [23]:
# (plays['absoluteYardlineNumber'] + plays['yardlineNumber']).max() # ok

In [24]:
# plays.query('yardlineNumber==50')['absoluteYardlineNumber'].unique() # ok

In [25]:
# plays.query('absoluteYardlineNumber>100').loc[23]

plays.query('absoluteYardlineNumber==108 and yardlineNumber==2 and possessionTeam=="BUF"').loc[2027]

gameId                                                                                     2022100901
playId                                                                                             96
playDescription            (14:49) D.Singletary right guard to BUF 2 for no gain (M.Jack, C.Heyward).
quarter                                                                                             1
down                                                                                                2
yardsToGo                                                                                          10
possessionTeam                                                                                    BUF
defensiveTeam                                                                                     PIT
yardlineSide                                                                                      BUF
yardlineNumber                                                                    

In [26]:
plays.pff_runConceptPrimary.unique()

array([nan, 'MAN', 'INSIDE ZONE', 'UNDEFINED', 'PULL LEAD',
       'OUTSIDE ZONE', 'TRAP', 'SNEAK', 'POWER', 'COUNTER', 'DRAW',
       'TRICK', 'FB RUN'], dtype=object)

In [27]:
# plays[['possessionTeam', 'yardlineSide', 'absoluteYardlineNumber', 'yardlineNumber']].sample(50)

In [28]:
def check_and_correct_yardline_data(data):

    counter = 0
    possession_team = data['possessionTeam']
    yardline_side = data['yardlineSide']  # чья сторона поля 
    absolute_yardline_number = data['absoluteYardlineNumber']
    yardline_number = data['yardlineNumber']
    
        
    if ((absolute_yardline_number < 0 ) or (absolute_yardline_number > 110)):
        raise ValueError("err")

    if possession_team == yardline_side:
        
        calculated_absolute_yardline = yardline_number + 10  
    else:
        calculated_absolute_yardline = 100 - yardline_number + 10
    
    if absolute_yardline_number != calculated_absolute_yardline:
        return True
    
    return False


In [29]:
plays.apply(check_and_correct_yardline_data, axis=1).sum()

7902

# 💊
We will introduce a new column that indicates the distance from the team's own end zone. The values will range from 0 to 100, making them easier to interpret. The absoluteYardlineNumber column will be removed!


In [30]:
def distance_from_own_endzone(row):
    if row['possessionTeam'] == row['yardlineSide']:
        return row['yardlineNumber']
    else:
        return 100 - row['yardlineNumber']

plays['distanceFromOwnEndZone'] = plays.apply(distance_from_own_endzone, axis=1)

In [31]:
plays['distanceFromOwnEndZone'].describe()

count    16124.000000
mean        48.921670
std         24.305607
min          1.000000
25%         28.000000
50%         45.000000
75%         68.000000
max         99.000000
Name: distanceFromOwnEndZone, dtype: float64

In [32]:
plays[plays['distanceFromOwnEndZone']==1][['playDescription', 'pff_manZone', 'possessionTeam', 'yardlineSide', 'yardlineNumber','down', 'yardsGained']]

,playDescription,pff_manZone,possessionTeam,yardlineSide,yardlineNumber,down,yardsGained
1313,(8:07) A.Dillon up the middle to GB 5 for 4 yards (M.Adams; N.Morrow).,Man,GB,GB,1,1,4
1822,(:17) (Shotgun) J.Winston pass deep right intended for C.Olave INTERCEPTED by J.Horn at 50. J.Ho...,Zone,NO,NO,1,1,0
1872,(12:38) J.Herbert pass incomplete short right to T.McKitty.,Zone,LAC,LAC,1,2,0
1886,(6:55) (Shotgun) C.Edwards-Helaire up the middle to KC 1 for no gain (G.Stewart).,Zone,KC,KC,1,1,0
2139,(1:18) (Shotgun) J.Allen pass deep middle to G.Davis to BUF 19 for 18 yards (J.Williams).,Zone,BUF,BUF,1,3,18
2381,(1:43) T.Tagovailoa up the middle to MIA 1 for no gain (D.Jones; T.Edmunds).,Zone,MIA,MIA,1,2,0
2534,(4:46) (Shotgun) M.Ryan pass deep right to A.Pierce to IND 31 for 30 yards (Ja.Watson).,Zone,IND,IND,1,1,30
2690,(7:58) D.Cook right guard to MIN 5 for 4 yards (D.Campbell; Q.Walker).,Zone,MIN,MIN,1,1,4
3118,(11:18) B.Robinson left guard to WAS 2 for 1 yard (R.Smith).,Zone,WAS,WAS,1,1,1
4956,(1:47) (Shotgun) Direct snap to N.Harris. N.Harris up the middle to PIT 4 for 3 yards (J.Hargra...,Zone,PIT,PIT,1,1,3


# Penalty ✅

## `penaltyYards`, `prePenaltyYardsGained` 



In [33]:
# sorted(list(plays.query("gameId==2022101604")['playId'].unique()))

In [34]:
# plays.query("gameId==2022101604 and playId==2791").T

In [35]:
# plays.query("gameId==2022101604 and playId==2861").T

In [36]:
len(plays) - plays['penaltyYards'].isna().sum()

384

In [37]:
def penalty(row):
    return True if 'penalty' in row.lower() else False

In [38]:

plays[plays['playDescription'].apply(penalty)]['penaltyYards'].isna().sum() #[['penaltyYards', 'prePenaltyYardsGained']]

0

In [39]:
len(plays[plays['playDescription'].apply(penalty)])

384

In [40]:
set(plays[plays['playDescription'].apply(penalty)]['penaltyYards'].index)==set(plays.query('~penaltyYards.isna()').index)


True

In [41]:
plays['penalty'] = plays['penaltyYards'].notna().astype(int)
plays['penaltyYards'] = plays['penaltyYards'].fillna(0)


# QB 

✅ `qbKneel`, `qbSneak`, `qbSpike`

### `qbKneel` ✅

In [42]:
def kneels(row):
    return True if 'kneels' in row.lower() else False


In [43]:
len(plays[plays['playDescription'].apply(kneels)]['qbKneel'])

164

In [44]:
len(plays.query('qbKneel==1'))

165

In [45]:
plays[(~plays['playDescription'].apply(kneels)) & (plays['qbKneel']==1)].T

,10816
gameId,2022092506
playId,4081
playDescription,(3:15) J.Burrow left end to NYJ 31 for -6 yards (D.Reed).
quarter,4
down,1
yardsToGo,10
possessionTeam,CIN
defensiveTeam,NYJ
yardlineSide,NYJ
yardlineNumber,25


In [46]:
plays.loc[10816]

gameId                                                                    2022092506
playId                                                                          4081
playDescription            (3:15) J.Burrow left end to NYJ 31 for -6 yards (D.Reed).
quarter                                                                            4
down                                                                               1
yardsToGo                                                                         10
possessionTeam                                                                   CIN
defensiveTeam                                                                    NYJ
yardlineSide                                                                     NYJ
yardlineNumber                                                                    25
gameClock                                                                      03:15
preSnapHomeScore                                                 

In [47]:
plays.loc[10816, 'qbKneel'] = 0

In [48]:
set(plays[plays['playDescription'].apply(kneels)]['qbKneel'].index)==set(plays.query('qbKneel==1').index)


True

In [49]:
plays.qbKneel.isna().sum()

0

### `qbSpike` ✅ 

In [50]:
def spikes(row):
    return True if 'spiked' in row.lower() else False


In [51]:
# len(plays[plays['playDescription'].apply(spikes)]['qbSpike'])

# len(plays.query('qbSpike==1'))

In [52]:
set(plays[plays['playDescription'].apply(spikes)]['qbSpike'].index)==set(plays.query('qbSpike==1').index)

True

In [53]:
plays.query('qbSpike.isna() and qbKneel != 1')[['playDescription', 'dropbackType']]

,playDescription,dropbackType
4,(2:16) (Shotgun) L.Jackson up the middle to TB 28 for -1 yards (R.Nunez-Roches).,DESIGNED_RUN
5,(14:15) Ja.Williams up the middle to DET 32 for 3 yards (C.Barton; U.Nwosu).,NaN
6,(:29) (Shotgun) J.Wilkins up the middle to IND 45 for 5 yards (J.Uche).,NaN
11,(12:55) (Shotgun) A.Dillon left tackle to WAS 46 for no gain (J.Smith-Williams; M.Sweat).,NaN
14,(9:41) (Shotgun) M.Mariota scrambles left end pushed ob at SEA 36 for 2 yards (J.Brooks).,SCRAMBLE_ROLLOUT_LEFT
...,...,...
16111,(2:17) (Shotgun) H.Haskins up the middle to TEN 38 for 3 yards (T.Johnson).,NaN
16115,"(2:49) J.Mixon up the middle to PIT 3 for -2 yards (T.Watt, A.Highsmith). PIT-A.Highsmith was in...",NaN
16118,"(13:41) (Shotgun) C.Patterson right tackle to NO 17 for 8 yards (M.Maye, T.Mathieu).",NaN
16121,(9:59) (Shotgun) P.Mahomes scrambles right end to IND 3 for 10 yards (R.Thomas).,SCRAMBLE


In [54]:
plays.query('qbSpike==1')[['isDropback', 'dropbackType']]

,isDropback,dropbackType
468,1,UNKNOWN
530,1,UNKNOWN
620,1,NaN
717,1,NaN
720,1,UNKNOWN
1271,1,NaN
1611,1,NaN
1658,1,NaN
2355,1,NaN
2501,1,UNKNOWN


In [55]:
plays.loc[plays['qbSpike'] == 1, 'dropbackType'] = plays.loc[plays['qbSpike'] == 1, 'dropbackType'].fillna("UNKNOWN")

plays[plays['qbSpike']==1]['dropbackType']

468      UNKNOWN
530      UNKNOWN
620      UNKNOWN
717      UNKNOWN
720      UNKNOWN
1271     UNKNOWN
1611     UNKNOWN
1658     UNKNOWN
2355     UNKNOWN
2501     UNKNOWN
2510     UNKNOWN
3135     UNKNOWN
3673     UNKNOWN
3910     UNKNOWN
5123     UNKNOWN
5262     UNKNOWN
5889     UNKNOWN
7047     UNKNOWN
9880     UNKNOWN
10812    UNKNOWN
12571    UNKNOWN
12704    UNKNOWN
13615    UNKNOWN
Name: dropbackType, dtype: object

In [56]:
plays.query('qbSpike==0')[['isDropback', 'dropbackType']].isna().sum()

isDropback      0
dropbackType    0
dtype: int64

In [57]:
plays[plays['playDescription'].apply(spikes) & (plays['qbSpike'].isna())]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,prePenaltyYardsGained,yardsGained,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,distanceFromOwnEndZone,penalty


In [58]:
plays['qbSpike'] = plays['qbSpike'].fillna(0).astype(int)

### `qbSneak` ✅

In [59]:

plays[plays[['qbSpike', 'qbKneel', 'qbSneak']].sum(axis=1)>=2]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,prePenaltyYardsGained,yardsGained,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,distanceFromOwnEndZone,penalty


In [60]:
plays['qbSneak'].isna().sum()

9336

In [61]:
plays.query('qbSneak==1')['pff_runConceptPrimary'].unique()

array(['SNEAK'], dtype=object)

In [62]:
plays.query('qbSneak==0')['pff_runConceptPrimary'].unique()

array(['MAN', 'INSIDE ZONE', 'UNDEFINED', 'PULL LEAD', 'OUTSIDE ZONE',
       nan, 'TRAP', 'POWER', 'COUNTER', 'DRAW', 'TRICK', 'FB RUN'],
      dtype=object)

In [63]:
plays.query('qbSneak.isna()')['pff_runConceptPrimary'].unique()

array([nan, 'OUTSIDE ZONE', 'INSIDE ZONE', 'MAN', 'UNDEFINED', 'DRAW',
       'POWER', 'COUNTER', 'TRICK', 'PULL LEAD', 'TRAP', 'FB RUN'],
      dtype=object)

In [64]:
plays.query('qbSneak.isna()')['playDescription'].head(7)

0          (1:54) (Shotgun) J.Burrow pass short middle to T.Boyd to CIN 30 for 9 yards (J.Hawkins).
1      (2:13) (Shotgun) J.Burrow pass short right to H.Hurst to CIN 12 for 4 yards (L.Vander Esch).
2           (2:00) (Shotgun) D.Mills pass short right to D.Pierce to HST 26 for 6 yards (D.Walker).
3     (9:28) (Shotgun) P.Mahomes pass short left to I.Pacheco to TEN 19 for 4 yards (Z.Cunningham).
8         (12:51) (Shotgun) J.Hurts pass incomplete short right to D.Smith (A.Bryant) [J.Cominsky].
9                (5:22) (Shotgun) C.Rush pass short left to M.Gallup to 50 for 15 yards (K.Fuller).
10                                (8:51) (Shotgun) C.Rush pass incomplete short right to D.Schultz.
Name: playDescription, dtype: object

Пасы?

In [65]:
def had_pass(row):
    return True if ' pass ' in row.lower() else False


In [66]:
len(plays[(plays['playDescription'].apply(had_pass)) & (plays['qbSneak'].isna())])

8705

In [67]:
plays.loc[(plays['playDescription'].apply(had_pass)) & (plays['qbSneak'].isna()), 'qbSneak'] = 0

In [68]:
plays.query('qbSneak.isna()')['playDescription']

90                                   (5:04) (Shotgun) M.Trubisky sacked at PIT 10 for -3 yards (J.Phillips).
101      (10:55) (Shotgun) M.Mariota sacked at SEA 36 for -7 yards (D.Taylor). FUMBLES (D.Taylor) [D.Tayl...
133                                          (:38) (Shotgun) D.Carr sacked at LV 12 for -7 yards (R.Weaver).
135                                     (12:49) (Shotgun) A.Rodgers sacked at GB 43 for -5 yards (D.Wonnum).
232                                              (13:33) B.Mayfield sacked at CAR 42 for -3 yards (D.Davis).
                                                        ...                                                 
16061                                      (:41) (Shotgun) Z.Wilson sacked at NYJ 14 for -5 yards (D.Jones).
16074                                        (14:21) (Shotgun) D.Carr sacked at LV 24 for -8 yards (J.Watt).
16082                                              (14:47) J.Fields sacked at CHI 15 for -5 yards (P.Smith).
16099              

In [69]:
def sacked(row):
    return True if 'sack' in row.lower() else False

len(plays.loc[(plays['playDescription'].apply(sacked))])

609

In [70]:
len(plays.loc[(plays['qbSneak'].isna()) & (plays['passResult']=="S") & (plays['playDescription'].apply(sacked))])

608

In [71]:
plays.loc[(plays['qbSneak'].isna()) & (plays['passResult']=="S") & (plays['playDescription'].apply(sacked)), 'qbSneak'] = 0

Оставшиеся:

In [72]:
plays[plays['qbSneak'].isna()][['playDescription', 'qbSpike']]

,playDescription,qbSpike
468,(:15) (No Huddle) J.Allen spiked the ball to stop the clock.,1
530,(:01) K.Cousins spiked the ball to stop the clock.,1
620,(:14) (No Huddle) M.Ryan spiked the ball to stop the clock.,1
717,(:09) M.Trubisky spiked the ball to stop the clock.,1
720,(:17) (No Huddle) J.Brissett spiked the ball to stop the clock.,1
1271,(:19) (No Huddle) T.Lawrence spiked the ball to stop the clock.,1
1611,(:37) (No Huddle) K.Murray spiked the ball to stop the clock.,1
1658,(1:01) (No Huddle) T.Brady spiked the ball to stop the clock.,1
2355,(:28) (No Huddle) K.Murray spiked the ball to stop the clock.,1
2501,(:04) (No Huddle) K.Cousins spiked the ball to stop the clock.,1


In [73]:
plays.loc[plays['qbSpike'] == 1, 'qbSneak'] = 0

In [74]:
plays['qbSneak'].isna().sum()

0

# Pass


`passLocationType`, `passResult`, `passLength`, `targetX`, `targetY`, `passTippedAtLine`

In [75]:
# plays.query('passResult.isna()')

plays.passResult.fillna('NA').value_counts()

NA    6388
C     5624
I     2911
S      608
R      400
IN     193
Name: passResult, dtype: int64

1. Check playdescription strings
2. Check other fields.

#### 1. Check Incomplete 



In [76]:
plays[plays['playDescription'].str.contains('incomplete', case=False)]['passResult'].value_counts()

I     2888
C        5
IN       1
Name: passResult, dtype: int64

In [77]:
# pd.set_option('max_colwidth', 300)


# plays[
#     (plays['playDescription'].str.contains('incomplete', case=False)) &
#     (plays['passResult'] != 'I')
# ]['playDescription'].to_frame()

# ok

In [78]:
plays.query('passResult=="C"')[['playDescription', 'passLength', 'targetX', 'targetY' ]].isna().sum()

playDescription    0
passLength         0
targetX            0
targetY            0
dtype: int64

In [79]:
completed = set(plays.query('passResult=="C"').index)  # 5624 штук

had_pass_set = set(plays.loc[(plays['playDescription'].apply(had_pass))].index)

completed.issubset(had_pass_set)

True

In [80]:
completed = (plays.loc[(plays['passResult']=="C") & (plays['playDescription'].apply(had_pass))])

plays[(plays['playDescription'].str.contains('incomplete', case=False)) & (plays['passResult'] == 'I')]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,prePenaltyYardsGained,yardsGained,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,distanceFromOwnEndZone,penalty
8,2022091104,3662,(12:51) (Shotgun) J.Hurts pass incomplete short right to D.Smith (A.Bryant) [J.Cominsky].,4,3,12,PHI,DET,PHI,35,...,0.0,0,1,NaN,NaN,0,Cover-0,Man,35,0
10,2022100912,445,(8:51) (Shotgun) C.Rush pass incomplete short right to D.Schultz.,1,3,5,DAL,LA,LA,15,...,0.0,0,1,NaN,NaN,0,Cover-3 Seam,Zone,85,0
17,2022102308,3304,"(2:57) (No Huddle, Shotgun) D.Mills pass incomplete short left to D.Ogunbowale.",4,2,1,HOU,LV,HOU,29,...,0.0,0,1,NaN,NaN,0,Quarters,Zone,29,0
23,2022092506,1412,(8:41) J.Burrow pass incomplete short right to J.Mixon [L.Joyner].,2,1,4,CIN,NYJ,NYJ,4,...,0.0,0,1,OUTSIDE ZONE,NaN,0,Cover-0,Man,96,0
30,2022091803,1222,(6:03) (Shotgun) M.Ryan pass incomplete short middle to D.Patmon.,2,3,6,IND,JAX,IND,26,...,0.0,0,1,NaN,NaN,0,2-Man,Man,26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16100,2022101611,742,(:53) (Shotgun) J.Hurts pass incomplete short right. Thrown away from outside the pocket.,1,2,12,PHI,DAL,DAL,18,...,0.0,0,1,OUTSIDE ZONE,NaN,1,Cover-1,Man,82,0
16103,2022092513,532,(7:19) (Shotgun) R.Wilson pass incomplete short middle to A.Okwuegbunam.,1,3,2,DEN,SF,DEN,33,...,0.0,0,1,NaN,NaN,0,Cover-2,Zone,33,0
16119,2022110604,1051,(12:49) (Shotgun) T.Lawrence pass incomplete short right to E.Engram.,2,3,4,JAX,LV,JAX,31,...,0.0,0,1,NaN,NaN,0,Cover-2,Zone,31,0
16120,2022103005,3492,(12:32) (Shotgun) K.Cousins pass incomplete deep right.,4,1,10,MIN,ARI,MIN,25,...,0.0,0,1,NaN,NaN,0,Cover-3,Zone,25,0


In [81]:
len(plays.loc[(plays['playDescription'].apply(sacked))]) == len(plays.loc[(plays['passResult']=="S")])


False

In [82]:
idx = list(set(plays.loc[(plays['playDescription'].apply(sacked))].index) - set(plays.loc[(plays['passResult']=="S")].index))
plays.loc[idx][['playDescription', 'passResult', 'passLength', 'targetX', 'targetY' ]]



,playDescription,passResult,passLength,targetX,targetY
3005,"(3:37) T.Heinicke sacked at GB 48 for -10 yards (K.Enagbare). FUMBLES (K.Enagbare) [K.Enagbare],...",I,-7.0,96.19,32.02


In [83]:
# plays.loc[(plays['playDescription'].str.contains('sacked', case=False))]['passResult'].fillna('NA').value_counts()

In [84]:
print(plays.loc[(plays['playDescription'].apply(kneels))]['passResult'].isna().sum())
print(plays.loc[(plays['playDescription'].apply(spikes))]['passResult'].isna().sum())
print(plays.loc[(plays['playDescription'].apply(had_pass))]['passResult'].isna().sum())

164
0
0


In [85]:
plays.loc[(plays['qbKneel']==1, 'passResult')] = "NoPass"

In [86]:
print(plays.loc[(plays['playDescription'].apply(kneels))]['passResult'].unique())
print(plays.loc[(plays['playDescription'].apply(spikes))]['passResult'].unique())
print(plays.loc[(plays['playDescription'].apply(had_pass))]['passResult'].unique())

['NoPass']
['I']
['C' 'I' 'IN' 'R']


In [87]:
plays.loc[(plays['playDescription'].apply(spikes))][['playDescription', 'passResult', 'timeToThrow', 
                                                     'timeInTackleBox', 'passLength', 'targetX', 
                                                     'dropbackType', 'targetY', 'dropbackDistance', 'yardsGained']]

,playDescription,passResult,timeToThrow,timeInTackleBox,passLength,targetX,dropbackType,targetY,dropbackDistance,yardsGained
468,(:15) (No Huddle) J.Allen spiked the ball to stop the clock.,I,NaN,NaN,0.0,NaN,UNKNOWN,NaN,NaN,0
530,(:01) K.Cousins spiked the ball to stop the clock.,I,NaN,NaN,0.0,NaN,UNKNOWN,NaN,NaN,0
620,(:14) (No Huddle) M.Ryan spiked the ball to stop the clock.,I,NaN,NaN,0.0,NaN,UNKNOWN,NaN,NaN,0
717,(:09) M.Trubisky spiked the ball to stop the clock.,I,NaN,NaN,0.0,NaN,UNKNOWN,NaN,NaN,0
720,(:17) (No Huddle) J.Brissett spiked the ball to stop the clock.,I,NaN,NaN,NaN,NaN,UNKNOWN,NaN,NaN,0
1271,(:19) (No Huddle) T.Lawrence spiked the ball to stop the clock.,I,NaN,NaN,NaN,NaN,UNKNOWN,NaN,NaN,0
1611,(:37) (No Huddle) K.Murray spiked the ball to stop the clock.,I,NaN,NaN,0.0,NaN,UNKNOWN,NaN,NaN,0
1658,(1:01) (No Huddle) T.Brady spiked the ball to stop the clock.,I,NaN,NaN,0.0,NaN,UNKNOWN,NaN,NaN,0
2355,(:28) (No Huddle) K.Murray spiked the ball to stop the clock.,I,NaN,NaN,0.0,NaN,UNKNOWN,NaN,NaN,0
2501,(:04) (No Huddle) K.Cousins spiked the ball to stop the clock.,I,NaN,NaN,0.0,NaN,UNKNOWN,NaN,NaN,0


# 💊


In [88]:
plays.loc[(plays['playDescription'].apply(spikes)), 'passResult'] = 'NoPass'

In [89]:
plays.loc[
    (plays['playDescription'].apply(had_pass)) & 
    (plays['dropbackType']=='SCRAMBLE')
][['playDescription', 'passResult', 'timeToThrow','timeInTackleBox', 'passLength',
   'targetX', 'dropbackType', 'targetY', 'dropbackDistance', 'yardsGained']]

,playDescription,passResult,timeToThrow,timeInTackleBox,passLength,targetX,dropbackType,targetY,dropbackDistance,yardsGained
12,(:35) (Shotgun) A.Rodgers pass deep left intended for R.Cobb INTERCEPTED by H.Smith [Z.Smith] at...,IN,4.506,3.5,45.0,75.33,SCRAMBLE,43.82,2.63,0
61,"(:23) (Shotgun) P.Walker pass deep left to D.Moore for 62 yards, TOUCHDOWN. PENALTY on CAR-D.Moo...",C,5.438,3.5,62.0,112.35,SCRAMBLE,40.71,4.59,62
113,(10:14) (Shotgun) M.Mariota pass short left to T.Allgeier to ATL 23 for 5 yards (D.Jackson).,C,4.104,3.8,-5.0,22.27,SCRAMBLE,33.89,2.61,5
138,(4:01) (Shotgun) J.Allen pass incomplete short left.,I,5.105,3.9,11.0,NaN,SCRAMBLE,NaN,2.93,0
151,(6:38) (Shotgun) J.Flacco pass incomplete deep right to E.Moore.,I,5.038,3.1,16.0,67.84,SCRAMBLE,53.17,2.14,0
...,...,...,...,...,...,...,...,...,...,...
16019,(9:30) (Shotgun) T.Lawrence pass incomplete short right.,I,5.739,3.2,0.0,NaN,SCRAMBLE,NaN,3.99,0
16028,(14:09) (Shotgun) J.Herbert pass incomplete short left to R.Rodgers (T.Andersen).,I,4.938,3.3,-7.0,71.72,SCRAMBLE,1.16,4.37,0
16048,(10:42) (Shotgun) J.Allen pass short right to Z.Moss to LA 26 for 8 yards (B.Wagner).,C,4.905,3.7,6.0,38.35,SCRAMBLE,45.54,1.75,8
16066,(13:23) (Shotgun) J.Fields pass incomplete short right to I.Smith-Marsette [K.Thibodeaux].,I,6.226,4.2,4.0,37.85,SCRAMBLE,-2.06,2.58,0


In [90]:
plays.passResult.fillna('NA').value_counts()

NA        6224
C         5624
I         2888
S          608
R          400
IN         193
NoPass     187
Name: passResult, dtype: int64

In [91]:
print(plays.loc[(plays['qbSneak']==1)]['passResult'].isna().sum())

plays.loc[(plays['qbSneak']==1)][['passResult', 'playDescription', 'rushLocationType', 'pff_runConceptPrimary']]

120


,passResult,playDescription,rushLocationType,pff_runConceptPrimary
44,NaN,(14:01) J.Brissett right guard to CLV 37 for 3 yards (L.Wilson).,INSIDE_RIGHT,SNEAK
47,NaN,(1:06) J.Hurts up the middle to DET 39 for 1 yard (D.Barnes).,INSIDE_RIGHT,SNEAK
49,NaN,"(3:10) T.Lawrence FUMBLES (Aborted) at JAX 34, RECOVERED by PHI-H.Reddick at JAX 35.",OUTSIDE_LEFT,SNEAK
81,NaN,(13:49) K.Cousins up the middle to MIN 37 for 3 yards (L.Fotu; V.Dimukeje).,INSIDE_RIGHT,SNEAK
106,NaN,(2:00) M.Trubisky up the middle to TB 26 for 3 yards (D.White).,INSIDE_RIGHT,SNEAK
...,...,...,...,...
15642,NaN,(11:29) D.Prescott up the middle pushed ob at CHI 18 for 25 yards (E.Jackson).,INSIDE_RIGHT,SNEAK
15868,NaN,(5:41) (No Huddle) J.Garoppolo up the middle to SF 37 for 3 yards (N.Bolton).,INSIDE_LEFT,SNEAK
15878,NaN,(5:48) T.Lawrence right guard to IND 40 for 2 yards (D.Buckner; T.Lewis).,INSIDE_RIGHT,SNEAK
15954,NaN,"(10:14) J.Hurts up the middle for 1 yard, TOUCHDOWN.",INSIDE_LEFT,SNEAK


In [92]:
plays.loc[(plays['qbSneak']==1) & (plays['passResult'].isna())]['rushLocationType'].unique()

array(['INSIDE_RIGHT', 'OUTSIDE_LEFT', 'OUTSIDE_RIGHT', 'INSIDE_LEFT'],
      dtype=object)

# 💊
### Sneak - "NoPass". 

In [93]:
plays.loc[(plays['qbSneak']==1), 'passResult'] = "NoPass"

In [94]:
plays.passResult.fillna('NA').value_counts()

NA        6104
C         5624
I         2888
S          608
R          400
NoPass     307
IN         193
Name: passResult, dtype: int64

In [95]:
plays.loc[(plays['playDescription'].apply(had_pass))]['passResult'].fillna('NA').value_counts()

C     5624
I     2888
IN     193
R        1
Name: passResult, dtype: int64

In [96]:
pd.set_option('max_colwidth', 300)

plays.loc[(plays['playDescription'].apply(had_pass)) & (plays['passResult']=='R')]['playDescription']

9898    (10:37) (Shotgun) J.Fields scrambles up the middle to CHI 23 for 3 yards. J.Fields pass short middle to E.St. Brown to GB 49 for 28 yards. PENALTY on CHI-J.Fields, Illegal Forward Pass, 5 yards, enforced at CHI 23.
Name: playDescription, dtype: object

In [97]:
pd.set_option('max_colwidth', 100)
plays.loc[(plays['playDescription'].apply(had_pass)) & (plays['passResult'].isna())]['playDescription']

Series([], Name: playDescription, dtype: object)

In [98]:
plays[['passResult', 'passLength', 'targetX', 'targetY']].isna().sum()

passResult    6104
passLength    7398
targetX       7748
targetY       7748
dtype: int64

In [99]:
plays[plays['passResult'].isna()][['playDescription', 'passLength', 'targetX', 'targetY']].isna().sum()

playDescription       0
passLength         6104
targetX            6104
targetY            6104
dtype: int64

In [100]:
plays.query('passResult.isna() and passLength.isna() and targetX.isna() and targetY.isna()')[['rushLocationType', 'pff_runConceptPrimary']].isna().sum()


rushLocationType          0
pff_runConceptPrimary    14
dtype: int64

In [101]:
plays.query('passResult.isna() and passLength.isna()'
            ' and targetX.isna() and targetY.isna()')[['playDescription', 'rushLocationType', 'pff_runConceptPrimary']]



,playDescription,rushLocationType,pff_runConceptPrimary
4,(2:16) (Shotgun) L.Jackson up the middle to TB 28 for -1 yards (R.Nunez-Roches).,INSIDE_LEFT,MAN
5,(14:15) Ja.Williams up the middle to DET 32 for 3 yards (C.Barton; U.Nwosu).,INSIDE_RIGHT,MAN
6,(:29) (Shotgun) J.Wilkins up the middle to IND 45 for 5 yards (J.Uche).,OUTSIDE_LEFT,INSIDE ZONE
11,(12:55) (Shotgun) A.Dillon left tackle to WAS 46 for no gain (J.Smith-Williams; M.Sweat).,INSIDE_LEFT,PULL LEAD
15,(3:18) J.Herron reported in as eligible. J.Jacobs left guard to DEN 3 for -2 yards (D.Jones).,OUTSIDE_LEFT,MAN
...,...,...,...
16109,(:36) S.Barkley right end to NYG 49 for -1 yards (C.Bryant).,OUTSIDE_RIGHT,PULL LEAD
16111,(2:17) (Shotgun) H.Haskins up the middle to TEN 38 for 3 yards (T.Johnson).,INSIDE_RIGHT,MAN
16115,"(2:49) J.Mixon up the middle to PIT 3 for -2 yards (T.Watt, A.Highsmith). PIT-A.Highsmith was in...",OUTSIDE_LEFT,MAN
16118,"(13:41) (Shotgun) C.Patterson right tackle to NO 17 for 8 yards (M.Maye, T.Mathieu).",OUTSIDE_LEFT,OUTSIDE ZONE


In [102]:
plays.query('passResult.isna() and passLength.isna()'
            ' and targetX.isna() and targetY.isna()'
            ' and rushLocationType!= "UNKNOWN"')[['playDescription', 'rushLocationType', 'pff_runConceptPrimary']]

,playDescription,rushLocationType,pff_runConceptPrimary
4,(2:16) (Shotgun) L.Jackson up the middle to TB 28 for -1 yards (R.Nunez-Roches).,INSIDE_LEFT,MAN
5,(14:15) Ja.Williams up the middle to DET 32 for 3 yards (C.Barton; U.Nwosu).,INSIDE_RIGHT,MAN
6,(:29) (Shotgun) J.Wilkins up the middle to IND 45 for 5 yards (J.Uche).,OUTSIDE_LEFT,INSIDE ZONE
11,(12:55) (Shotgun) A.Dillon left tackle to WAS 46 for no gain (J.Smith-Williams; M.Sweat).,INSIDE_LEFT,PULL LEAD
15,(3:18) J.Herron reported in as eligible. J.Jacobs left guard to DEN 3 for -2 yards (D.Jones).,OUTSIDE_LEFT,MAN
...,...,...,...
16109,(:36) S.Barkley right end to NYG 49 for -1 yards (C.Bryant).,OUTSIDE_RIGHT,PULL LEAD
16111,(2:17) (Shotgun) H.Haskins up the middle to TEN 38 for 3 yards (T.Johnson).,INSIDE_RIGHT,MAN
16115,"(2:49) J.Mixon up the middle to PIT 3 for -2 yards (T.Watt, A.Highsmith). PIT-A.Highsmith was in...",OUTSIDE_LEFT,MAN
16118,"(13:41) (Shotgun) C.Patterson right tackle to NO 17 for 8 yards (M.Maye, T.Mathieu).",OUTSIDE_LEFT,OUTSIDE ZONE


In [103]:
plays[plays['playDescription'].str.contains('scrambles', case=False, na=False)]['passResult'].unique()

array(['R'], dtype=object)

In [104]:
pd.set_option('max_colwidth', 300)
plays[(plays['passResult'].isna()) & (plays['playDescription'].str.contains('lateral|backward', case=False, na=False))]['playDescription']

4148    (:25) (No Huddle, Shotgun) B.Bolden left end to LV 23 for 5 yards. Lateral to F.Moreau to LV 28 for 5 yards (T.Herndon). FUMBLES (T.Herndon), RECOVERED by JAX-D.Lloyd at LV 25. D.Lloyd to LV 14 for 11 yards (D.Parham).
Name: playDescription, dtype: object

In [105]:
plays.loc[(plays['playDescription'].str.contains('touchdown', case=False))][['playDescription', 'qbSneak', 'passResult',
                                                                             'rushLocationType', 'pff_runConceptPrimary', 
                                                                             'passLocationType', 'passLength', 'targetX',
                                                                             'targetY']]



,playDescription,qbSneak,passResult,rushLocationType,pff_runConceptPrimary,passLocationType,passLength,targetX,targetY
29,"(8:54) C.Van Lanen reported in as eligible. T.Lawrence pass short right to J.Robinson for 3 yards, TOUCHDOWN.",0,C,NaN,MAN,OUTSIDE_RIGHT,3.0,10.10,44.41
34,"(3:39) M.Stafford pass short left to Al.Robinson for 5 yards, TOUCHDOWN.",0,C,NaN,MAN,INSIDE_BOX,5.0,112.34,46.03
40,"(4:49) T.Allgeier left guard for 1 yard, TOUCHDOWN.",False,NaN,OUTSIDE_LEFT,MAN,NaN,NaN,NaN,NaN
61,"(:23) (Shotgun) P.Walker pass deep left to D.Moore for 62 yards, TOUCHDOWN. PENALTY on CAR-D.Moore, Unsportsmanlike Conduct, 15 yards, enforced between downs.",0,C,NaN,NaN,OUTSIDE_LEFT,62.0,112.35,40.71
67,"(7:46) (Shotgun) R.Tannehill pass short middle to C.Okonkwo for 8 yards, TOUCHDOWN. IND-S.Leonard was injured during the play.",0,C,NaN,NaN,INSIDE_BOX,6.0,107.35,23.54
...,...,...,...,...,...,...,...,...,...
16063,"(9:43) (Shotgun) P.Mahomes pass short left to T.Kelce for 9 yards, TOUCHDOWN [M.Golden].",0,C,NaN,OUTSIDE ZONE,INSIDE_BOX,9.0,6.65,4.74
16081,"(9:21) K.Cousins pass short right to J.Jefferson for 5 yards, TOUCHDOWN.",0,C,NaN,INSIDE ZONE,INSIDE_BOX,3.0,108.39,5.10
16084,"(14:27) J.Goff pass short middle to J.Reynolds for 3 yards, TOUCHDOWN. Penalty on SEA-M.Jackson, Defensive Holding, declined.",0,C,NaN,INSIDE ZONE,INSIDE_BOX,3.0,4.90,29.59
16097,"(1:10) (Shotgun) J.Dobbins left tackle for 4 yards, TOUCHDOWN.",False,NaN,INSIDE_LEFT,MAN,NaN,NaN,NaN,NaN


In [106]:
plays['passResult'].isna().sum()

6104

In [107]:
len(plays[
    (plays['passResult'].isna()) &
    (plays['passLength'].isna()) & (plays['targetX'].isna()) & (plays['targetY'].isna())
])

6104

In [108]:
plays[
    (plays['passResult'].isna()) &
    (plays['passLength'].isna()) & (plays['targetX'].isna()) & (plays['targetY'].isna())
]['isDropback'].unique()


array([0])

In [109]:
plays['passResult'] = plays['passResult'].fillna("NoPass")

In [110]:
 plays[
    (plays['passResult'].isna()) &
    (plays['playDescription'].str.contains(' pass', case=False, na=False))
]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,prePenaltyYardsGained,yardsGained,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,distanceFromOwnEndZone,penalty


In [111]:
plays.loc[(plays['qbSneak']==1) | (plays['qbKneel']==1) | (plays['qbSpike']==1)]['passResult'].unique()

array(['NoPass'], dtype=object)

### `passLocationType` ✅


6812 missing values

The location type of where the QB was at the time of throw (Inside Tackle Box, Outside Left, Outside Right, Unknown, text)

In [112]:
plays['passLocationType'].fillna('NA').value_counts()

INSIDE_BOX       8018
NA               6812
OUTSIDE_RIGHT     811
OUTSIDE_LEFT      482
UNKNOWN             1
Name: passLocationType, dtype: int64

In [113]:
location_analysis = plays.groupby('passLocationType').agg({
    'timeInTackleBox': 'mean',  
    'timeToThrow': 'mean', 'dropbackType': lambda x: x.value_counts().to_dict(),  #
    'passResult': lambda x: x.value_counts().to_dict(), }).reset_index()


location_analysis


,passLocationType,timeInTackleBox,timeToThrow,dropbackType,passResult
0,INSIDE_BOX,2.528538,2.531454,"{'TRADITIONAL': 7804, 'DESIGNED_ROLLOUT_RIGHT': 124, 'DESIGNED_ROLLOUT_LEFT': 87, 'DESIGNED_RUN': 2, 'SCRAMBLE_ROLLOUT_RIGHT': 1}","{'C': 5111, 'I': 2355, 'S': 390, 'IN': 162}"
1,OUTSIDE_LEFT,3.524421,4.420753,"{'SCRAMBLE': 184, 'TRADITIONAL': 179, 'DESIGNED_ROLLOUT_LEFT': 102, 'SCRAMBLE_ROLLOUT_LEFT': 16, 'DESIGNED_RUN': 1}","{'C': 180, 'I': 177, 'S': 117, 'IN': 8}"
2,OUTSIDE_RIGHT,3.100000,4.186400,"{'SCRAMBLE': 318, 'DESIGNED_ROLLOUT_RIGHT': 244, 'TRADITIONAL': 165, 'SCRAMBLE_ROLLOUT_RIGHT': 81, 'DESIGNED_RUN': 2, 'DESIGNED_ROLLOUT_LEFT': 1}","{'I': 355, 'C': 332, 'S': 101, 'IN': 23}"
3,UNKNOWN,2.436000,2.436000,{'TRADITIONAL': 1},{'I': 1}


In [114]:
# plays.query('passLocationType=="UNKNOWN"').T

In [115]:
plays.loc[ (plays['passLocationType'] == 'UNKNOWN'), 'passLocationType'] = 'INSIDE_BOX'

In [117]:
plays.groupby('passResult')['passLocationType'].unique().to_frame()

,passLocationType
passResult,
C,"[INSIDE_BOX, OUTSIDE_RIGHT, OUTSIDE_LEFT, nan]"
I,"[INSIDE_BOX, OUTSIDE_RIGHT, OUTSIDE_LEFT]"
IN,"[OUTSIDE_LEFT, OUTSIDE_RIGHT, INSIDE_BOX]"
NoPass,[nan]
R,[nan]
S,"[OUTSIDE_LEFT, INSIDE_BOX, OUTSIDE_RIGHT]"


In [118]:
plays.loc[
    (plays['timeToThrow'] > plays['timeInTackleBox'] + 1e-6) & (plays['passLocationType']=='INSIDE_BOX')

][['timeToThrow', 'timeInTackleBox', 'passResult', 'playDescription']]

,timeToThrow,timeInTackleBox,passResult,playDescription
621,3.470,2.0,I,"(2:15) C.Wentz pass incomplete short right [D.Wilson]. PENALTY on WAS-C.Wentz, Intentional Grounding, 14 yards, enforced at DAL 17."
629,3.904,1.9,C,(2:50) M.Mariota pass deep right to O.Zaccheaus to SEA 39 for 23 yards (T.Woolen) [P.Ford].
4130,3.337,1.6,I,(10:32) J.Allen pass incomplete deep right to R.Gilliam.
4741,4.204,2.9,I,(6:46) (Shotgun) M.Mariota pass incomplete short left to K.Pitts (G.Delpit).
5830,6.774,3.4,I,(8:34) R.Wilson pass incomplete short right to J.Williams (J.Adams).
7060,6.239,2.9,I,(1:22) (Shotgun) L.Jackson pass incomplete deep middle to D.Robinson (J.Love).
8001,3.970,3.1,I,(6:14) (Shotgun) T.Tagovailoa pass incomplete short left to T.Hill.
8249,4.638,2.5,C,(10:57) (Shotgun) D.Jones pass short left to R.James pushed ob at JAX 35 for 2 yards (A.Key).
9093,4.505,3.3,I,(8:48) (Shotgun) M.Mariota pass incomplete deep left to K.Pitts.
9174,4.137,2.1,I,(4:47) (Shotgun) M.Mariota pass incomplete deep left to K.Pitts.


In [119]:
# plays.loc[
#     (plays['timeToThrow'] > plays['timeInTackleBox'] + 1e-6) & (plays['passLocationType']=='INSIDE_BOX'), 'passLocationType'] = np.NaN

In [120]:
plays.loc[plays['passResult'].isin(['NoPass', 'R']), 'passLocationType'] = 'NoPass'

In [121]:
# plays.query('dropbackType in ("DESIGNED_RUN", "QB_SNEAK")')['passLocationType'].unique()

plays.query('passLocationType.isna()')[['passResult', 'playDescription', 'qbSpike',
                                        'timeToThrow', 'timeInTackleBox', 'passLength', 'dropbackType', 'dropbackDistance' ]]

,passResult,playDescription,qbSpike,timeToThrow,timeInTackleBox,passLength,dropbackType,dropbackDistance
7043,C,"(12:42) (Shotgun) B.Berrios pass short right to Z.Wilson for 2 yards, TOUCHDOWN.",0,3.27,0.0,2.0,DESIGNED_ROLLOUT_RIGHT,1.22


In [122]:
plays.loc[7043, 'passLocationType'] = 'OUTSIDE_RIGHT'

In [123]:
plays['passLocationType'].isna().sum() 

0

### `passLength` ✅

7398 gaps

In [124]:
plays[plays['passLength']==0][['playDescription', 'passResult']].head()

,playDescription,passResult
54,(14:52) (Shotgun) M.Ryan pass incomplete short right to J.Taylor.,I
125,(14:56) B.Mayfield pass incomplete short right to R.Anderson (J.Clowney).,I
140,(5:13) K.Pickett pass short right to Z.Gentry to PHI 39 for 3 yards (K.White).,C
185,(3:52) (Shotgun) A.Rodgers pass incomplete short right to C.Watson.,I
239,"(:44) (No Huddle, Shotgun) J.Brissett pass short left to N.Chubb to NE 23 for 14 yards (D.Ekuale).",C


In [125]:
plays[plays['passResult']=="NoPass"]['passLength'].unique()

array([nan,  0., -1.,  1.])

In [126]:
plays.loc[plays['passResult']=="NoPass", 'passLength'] = 0

In [127]:
plays[plays['passLength'].isna()]['passResult'].unique()

array(['R', 'S'], dtype=object)

In [128]:
plays.loc[plays['passResult']=="S", 'passLength'] = 0

In [129]:
plays.loc[plays['passResult']=="R", 'passLength'].unique()

array([nan,  0.])

In [130]:
plays[
    (plays['playDescription'].str.contains('scramble', case=False)) &
    (~plays['passLength'].isna())
]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,prePenaltyYardsGained,yardsGained,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,distanceFromOwnEndZone,penalty
9898,2022091812,1128,"(10:37) (Shotgun) J.Fields scrambles up the middle to CHI 23 for 3 yards. J.Fields pass short middle to E.St. Brown to GB 49 for 28 yards. PENALTY on CHI-J.Fields, Illegal Forward Pass, 5 yards, enforced at CHI 23.",2,3,10,CHI,GB,CHI,20,...,31.0,-2,1,NaN,NaN,0,Cover 6-Left,Zone,20,1


In [131]:
plays.loc[plays['passResult']=="R", 'passLength'] = 0

In [132]:
plays['passAfterScramble'] = (
    ((plays['passResult'] == 'R') | (plays['dropbackType'].str.contains('scramble', case=False, na=False))) &
    (plays['targetX'].notna() | plays['targetY'].notna() | plays['playDescription'].str.contains(' pass', case=False, na=False))
).astype(int)

In [133]:
plays.query('passAfterScramble==1')[['passResult', 'dropbackType', 'timeToThrow', 'timeInTackleBox', 'targetX', 'targetY', 'playDescription']]

,passResult,dropbackType,timeToThrow,timeInTackleBox,targetX,targetY,playDescription
12,IN,SCRAMBLE,4.506,3.5,75.33,43.82,(:35) (Shotgun) A.Rodgers pass deep left intended for R.Cobb INTERCEPTED by H.Smith [Z.Smith] at MIN 30. H.Smith to MIN 30 for no gain (R.Cobb).
61,C,SCRAMBLE,5.438,3.5,112.35,40.71,"(:23) (Shotgun) P.Walker pass deep left to D.Moore for 62 yards, TOUCHDOWN. PENALTY on CAR-D.Moore, Unsportsmanlike Conduct, 15 yards, enforced between downs."
113,C,SCRAMBLE,4.104,3.8,22.27,33.89,(10:14) (Shotgun) M.Mariota pass short left to T.Allgeier to ATL 23 for 5 yards (D.Jackson).
138,I,SCRAMBLE,5.105,3.9,NaN,NaN,(4:01) (Shotgun) J.Allen pass incomplete short left.
151,I,SCRAMBLE,5.038,3.1,67.84,53.17,(6:38) (Shotgun) J.Flacco pass incomplete deep right to E.Moore.
...,...,...,...,...,...,...,...
16019,I,SCRAMBLE,5.739,3.2,NaN,NaN,(9:30) (Shotgun) T.Lawrence pass incomplete short right.
16028,I,SCRAMBLE,4.938,3.3,71.72,1.16,(14:09) (Shotgun) J.Herbert pass incomplete short left to R.Rodgers (T.Andersen).
16048,C,SCRAMBLE,4.905,3.7,38.35,45.54,(10:42) (Shotgun) J.Allen pass short right to Z.Moss to LA 26 for 8 yards (B.Wagner).
16066,I,SCRAMBLE,6.226,4.2,37.85,-2.06,(13:23) (Shotgun) J.Fields pass incomplete short right to I.Smith-Marsette [K.Thibodeaux].


In [134]:
plays['passLength'].isna().sum()

0

### `targetX`, `targetY` ✅

In [135]:
plays.groupby('passResult')[['targetX', 'targetY']].agg(lambda x: x.isna().sum())

,targetX,targetY
passResult,,
C,0,0
I,329,329
IN,0,0
NoPass,6411,6411
R,400,400
S,608,608


### `passTippedAtLine` ✅

In [136]:
plays.passTippedAtLine.value_counts(dropna=False)

False    9109
NaN      6788
True      227
Name: passTippedAtLine, dtype: int64

In [137]:
plays.groupby(['passTippedAtLine', 'passResult'])[['targetX', 'targetY']].agg(lambda x: x.isna().sum())

targetX  targetY
passTippedAtLine passResult                  
False            C                 0        0
                 I               328      328
                 IN                0        0
                 NoPass           23       23
                 S               608      608
True             C                 0        0
                 I                 1        1
                 IN                0        0

In [138]:
plays.query('passTippedAtLine.isna()').groupby('passResult')[['targetX', 'targetY']].agg(lambda x: x.isna().sum())

,targetX,targetY
passResult,,
NoPass,6388,6388
R,400,400


In [139]:
# plays.loc[(plays['passTippedAtLine'].isna()) & (plays['passResult']=="R"), 'passLength'] = False


plays['passTippedAtLine'] = plays['passTippedAtLine'].fillna(False).astype(int)

# Dropback 


`isDropback`, `dropbackType`, `dropbackDistance`


✅ `isDropback` -- no missing values

✅ `dropbackType` -- T

['TRADITIONAL', 'DESIGNED_RUN', nan, 'SCRAMBLE', 'SCRAMBLE_ROLLOUT_LEFT', 'DESIGNED_ROLLOUT_RIGHT', 'DESIGNED_ROLLOUT_LEFT', 'QB_SNEAK', 'UNKNOWN', 'SCRAMBLE_ROLLOUT_RIGHT']

In [140]:
plays.groupby('isDropback')['passResult'].unique()

isDropback
0                    [NoPass]
1    [C, I, IN, R, S, NoPass]
Name: passResult, dtype: object

In [141]:
plays.query('isDropback==1')[['dropbackType']].isna().sum()

dropbackType    1
dtype: int64

In [142]:
plays.query('isDropback==0')[['dropbackType']].isna().sum()

dropbackType    5790
dtype: int64

In [143]:
plays.query('isDropback==1 and dropbackType.isna()').T

,903
gameId,2022091108
playId,3396
playDescription,(3:27) (Shotgun) K.Toney scrambles left end to TEN 22 for 4 yards (K.Fulton).
quarter,4
down,1
yardsToGo,10
possessionTeam,NYG
defensiveTeam,TEN
yardlineSide,TEN
yardlineNumber,26


In [144]:
plays.loc[903, 'dropbackType'] = 'DESIGNED_ROLLOUT_RIGHT'

Fill in the gaps with the "UNKNOWN" value? 



In [145]:
plays.query('dropbackType=="UNKNOWN"')[['playDescription', 'isDropback', 'qbSpike', 'qbKneel']]

,playDescription,isDropback,qbSpike,qbKneel
53,(1:16) J.Brissett kneels to CIN 10 for no gain.,0,0,1
121,(4:05) C.Edmonds up the middle to MIA 9 for 1 yard (D.Wise).,0,0,0
173,(1:31) C.Rush kneels to DAL 38 for -1 yards.,0,0,1
180,(:05) R.Wilson kneels to DEN 24 for -1 yards.,0,0,1
222,(3:38) (Shotgun) D.Montgomery left guard to SF 44 for 1 yard (A.Al-Shaair; C.Ward).,0,0,0
...,...,...,...,...
13554,"(1:32) (Shotgun) A.Rodgers Aborted. J.Myers FUMBLES at GB 43, recovered by GB-A.Dillon at GB 33.",0,0,0
13615,"(:38) (No Huddle) J.Winston spiked the ball to stop the clock. PENALTY on NO-J.Winston, Intentional Grounding, 10 yards, enforced at ATL 40.",1,1,0
13671,(:11) J.Herbert kneels to LAC 34 for -1 yards.,0,0,1
13727,(:06) A.Rodgers kneels to GB 15 for -1 yards.,0,0,1


In [147]:
plays.query('~dropbackType.isna() and qbKneel==1')[['dropbackType', 'playDescription', 'isDropback', 'offenseFormation']].head(10)


,dropbackType,playDescription,isDropback,offenseFormation
53,UNKNOWN,(1:16) J.Brissett kneels to CIN 10 for no gain.,0,NaN
173,UNKNOWN,(1:31) C.Rush kneels to DAL 38 for -1 yards.,0,NaN
180,UNKNOWN,(:05) R.Wilson kneels to DEN 24 for -1 yards.,0,NaN
286,UNKNOWN,(:09) G.Smith kneels to DEN 49 for -1 yards.,0,NaN
340,UNKNOWN,(1:09) M.Jones kneels to NYJ 47 for -2 yards.,0,NaN
607,UNKNOWN,(:25) S.Ehlinger kneels to IND 13 for -1 yards.,0,NaN
612,UNKNOWN,(:15) A.Rodgers kneels to GB 24 for -1 yards.,0,NaN
1356,UNKNOWN,(:18) M.Jones kneels to NE 23 for -2 yards.,0,NaN
1584,UNKNOWN,(1:01) J.Fields kneels to HST 11 for no gain.,0,NaN
1666,UNKNOWN,(1:13) R.Tannehill kneels to LV 43 for -1 yards.,0,NaN


In [148]:
plays.query('dropbackType.isna() and qbKneel==1')[['dropbackType', 'playDescription', 'isDropback', 'offenseFormation']].head(10)

,dropbackType,playDescription,isDropback,offenseFormation
7,NaN,(:35) K.Murray kneels to CAR 29 for -1 yards.,0,NaN
25,NaN,(:10) G.Smith kneels to SEA 24 for -1 yards.,0,NaN
197,NaN,(:53) J.Herbert kneels to LV 23 for -1 yards.,0,NaN
224,NaN,(:27) J.Fields kneels to NE 3 for -1 yards.,0,NaN
393,NaN,(:11) T.Lawrence kneels to JAX 20 for -1 yards.,0,NaN
476,NaN,(1:30) C.Henne kneels to KC 19 for -1 yards.,0,NaN
498,NaN,(:25) G.Smith kneels to SEA 8 for -1 yards.,0,NaN
579,NaN,(:29) J.Burrow kneels to CIN 43 for -1 yards.,0,NaN
623,NaN,(:11) J.Brissett kneels to CLV 24 for -1 yards.,0,NaN
635,NaN,(:15) B.Mayfield kneels to CAR 24 for -1 yards.,0,NaN


In [149]:
print(plays.query('qbKneel==1')['dropbackType'].unique())
print(plays.query('qbKneel==1')[ 'isDropback'].unique())

[nan 'UNKNOWN']
[0]


In [150]:
plays.loc[plays['qbKneel'] == 1, 'dropbackType'] = plays.loc[plays['qbKneel'] == 1, 'dropbackType'].fillna('UNKNOWN')

In [151]:
print(plays.query('qbSpike==1')['dropbackType'].unique())
print(plays.query('qbSpike==1')[ 'isDropback'].unique())

['UNKNOWN']
[1]


In [152]:
plays.query('isDropback==0')[ 'dropbackType'].unique()

array(['DESIGNED_RUN', nan, 'UNKNOWN', 'QB_SNEAK',
       'SCRAMBLE_ROLLOUT_RIGHT', 'SCRAMBLE', 'SCRAMBLE_ROLLOUT_LEFT'],
      dtype=object)

In [153]:
plays.dropbackType.unique()

array(['TRADITIONAL', 'DESIGNED_RUN', nan, 'UNKNOWN', 'SCRAMBLE',
       'SCRAMBLE_ROLLOUT_LEFT', 'DESIGNED_ROLLOUT_RIGHT',
       'DESIGNED_ROLLOUT_LEFT', 'QB_SNEAK', 'SCRAMBLE_ROLLOUT_RIGHT'],
      dtype=object)

In [154]:
plays.query('isDropback==0 and ~dropbackType.isna() and dropbackType!= "UNKNOWN"')[['dropbackType', 'playDescription',
                                                                                    'isDropback', 'offenseFormation']]


,dropbackType,playDescription,isDropback,offenseFormation
4,DESIGNED_RUN,(2:16) (Shotgun) L.Jackson up the middle to TB 28 for -1 yards (R.Nunez-Roches).,0,PISTOL
44,QB_SNEAK,(14:01) J.Brissett right guard to CLV 37 for 3 yards (L.Wilson).,0,SINGLEBACK
47,QB_SNEAK,(1:06) J.Hurts up the middle to DET 39 for 1 yard (D.Barnes).,0,I_FORM
58,DESIGNED_RUN,"(12:41) (Shotgun) L.Jackson right tackle to BLT 40 for 4 yards (C.Mosley, J.Whitehead).",0,EMPTY
64,DESIGNED_RUN,"(11:47) (No Huddle, Shotgun) M.Mariota right tackle to SF 15 for 5 yards (K.Hyder).",0,SHOTGUN
...,...,...,...,...
16011,DESIGNED_RUN,"(5:22) (No Huddle, Shotgun) J.Hurts left end to HST 4 for no gain (J.Owens).",0,PISTOL
16071,DESIGNED_RUN,(4:44) (Shotgun) P.Mahomes left end to TB 3 for 9 yards (L.Ryan).,0,SHOTGUN
16072,DESIGNED_RUN,(9:02) (Shotgun) S.Ehlinger left guard to NE 40 for 2 yards (J.Tavai).,0,SHOTGUN
16079,DESIGNED_RUN,(1:41) 7-T.Hill at quarterback. (Shotgun) T.Hill right end to CIN 19 for -1 yards (E.Apple).,0,WILDCAT


In [155]:
plays.query('isDropback==0 and ~dropbackType.isna() and dropbackType!= "UNKNOWN"')['dropbackType'].unique()


array(['DESIGNED_RUN', 'QB_SNEAK', 'SCRAMBLE_ROLLOUT_RIGHT', 'SCRAMBLE',
       'SCRAMBLE_ROLLOUT_LEFT'], dtype=object)

In [156]:
plays.query('isDropback==1 and ~dropbackType.isna() and dropbackType!= "UNKNOWN"')['dropbackType'].unique()


array(['TRADITIONAL', 'SCRAMBLE', 'SCRAMBLE_ROLLOUT_LEFT',
       'DESIGNED_ROLLOUT_RIGHT', 'DESIGNED_ROLLOUT_LEFT',
       'SCRAMBLE_ROLLOUT_RIGHT', 'DESIGNED_RUN'], dtype=object)

In [157]:
plays['dropbackType'] = plays['dropbackType'].fillna("UNKNOWN")

### `dropbackDistance` ✅

### `playAction` ✅

In [158]:
plays.playAction.isna().sum()

0

### `unblockedPressure` ✅



In [159]:
plays.unblockedPressure.value_counts(dropna=False)

False    9051
NaN      6369
True      704
Name: unblockedPressure, dtype: int64

### `timeToThrow`, `timeInTackleBox` ✅

In [160]:
plays.query('timeToThrow.isna()')['passResult'].value_counts()

NoPass    6411
S          608
R          400
Name: passResult, dtype: int64

In [161]:
plays.query('timeInTackleBox.isna()')['passResult'].value_counts()

NoPass    6411
R          400
S          396
Name: passResult, dtype: int64

### `timeToSack` ✅

In [162]:
# plays.query('timeToSack.isna()')['passResult'].value_counts()

In [163]:
# plays.query('~timeToSack.isna()')['passResult'].value_counts()

# Defence

## `pff_passCoverage`

In [164]:
plays['pff_passCoverage']

0         Cover-3
1        Quarters
2        Quarters
3        Quarters
4         Cover-1
           ...   
16119     Cover-2
16120     Cover-3
16121    Quarters
16122     Cover-3
16123     Cover-3
Name: pff_passCoverage, Length: 16124, dtype: object

# Offense

## `playAction` ✅ 

### `pff_runConceptPrimary`, `pff_runConceptSecondary` ✅

In [169]:
plays.pff_runConceptPrimary.unique()

array([nan, 'MAN', 'INSIDE ZONE', 'UNDEFINED', 'PULL LEAD',
       'OUTSIDE ZONE', 'TRAP', 'SNEAK', 'POWER', 'COUNTER', 'DRAW',
       'TRICK', 'FB RUN'], dtype=object)

In [170]:
plays.pff_runConceptSecondary.unique()

array([nan, 'READ OPTION', 'LEAD', 'PITCH', 'INVERTED;READ OPTION',
       'SPLIT', 'MISDIRECTION', 'READ OPTION;SPLIT', 'LEAD;PITCH',
       'CROSS LEAD;LEAD', 'CROSS LEAD', 'LEAD;SPLIT', 'PITCH;SPLIT',
       'READ OPTION;SPEED OPTION', 'LEAD;READ OPTION;SPLIT',
       'LEAD;QB RUNS', 'QB RUNS', 'LEAD;READ OPTION',
       'CROSS LEAD;READ OPTION', 'CROSS LEAD;QB RUNS',
       'MISDIRECTION;PITCH', 'LEAD;SPEED OPTION',
       'READ OPTION;SPEED OPTION;SPLIT', 'INVERTED;READ OPTION;SPLIT',
       'SPEED OPTION', 'BACKSIDE FOLD',
       'INVERTED;READ OPTION;SPEED OPTION',
       'CROSS LEAD;READ OPTION;SPLIT', 'CROSS LEAD;LEAD;READ OPTION',
       'CROSS LEAD;INVERTED;READ OPTION', 'CROSS LEAD;SPLIT',
       'CROSS LEAD;PITCH', 'CROSS LEAD;MISDIRECTION',
       'MISDIRECTION;SPLIT', 'CROSS LEAD;READ OPTION;SPEED OPTION',
       'LEAD;MISDIRECTION', 'QB RUNS;SPLIT', 'MISDIRECTION;QB RUNS',
       'LEAD;READ OPTION;SPEED OPTION', 'LEAD;PITCH;SPLIT',
       'CROSS LEAD;LEAD;QB RUNS', 'BAC

In [171]:
plays.query('pff_runConceptPrimary=="SNEAK"')['qbSneak'].value_counts(dropna=False)

True    120
Name: qbSneak, dtype: int64

In [172]:
plays.qbSneak.value_counts(dropna=False)

0       16004
True      120
Name: qbSneak, dtype: int64

In [173]:
plays.query('pff_runConceptPrimary=="TRICK"')['playAction'].value_counts(dropna=False)

0    186
1     67
Name: playAction, dtype: int64

In [174]:
plays.query('pff_runConceptPrimary=="TRICK"')['pff_runConceptSecondary']

194        NaN
198        NaN
249        NaN
251      SPLIT
279       LEAD
         ...  
15844      NaN
15869      NaN
15898      NaN
15918    SPLIT
15975      NaN
Name: pff_runConceptSecondary, Length: 253, dtype: object

In [175]:
plays['pff_runConceptPrimary'] = plays['pff_runConceptPrimary'].fillna("UNKNOWN")
plays['pff_runConceptSecondary'] = plays['pff_runConceptSecondary'].fillna("UNKNOWN")

### `rushLocationType` ✅



In [176]:
plays.rushLocationType.unique()

array([nan, 'INSIDE_LEFT', 'INSIDE_RIGHT', 'OUTSIDE_LEFT', 'UNKNOWN',
       'OUTSIDE_RIGHT'], dtype=object)

In [177]:
plays['rushLocationType'] = plays['rushLocationType'].fillna("UNKNOWN")


## `offenseFormation`, `playDescription`

`offenseFormation`, `playDescription` -- different formations? 

In [178]:
plays[['offenseFormation', 'playDescription', 'passResult', 'dropbackType', 'dropbackDistance', 'playAction', 'qbKneel']].head(5)

,offenseFormation,playDescription,passResult,dropbackType,dropbackDistance,playAction,qbKneel
0,EMPTY,(1:54) (Shotgun) J.Burrow pass short middle to T.Boyd to CIN 30 for 9 yards (J.Hawkins).,C,TRADITIONAL,2.40,0,0
1,EMPTY,(2:13) (Shotgun) J.Burrow pass short right to H.Hurst to CIN 12 for 4 yards (L.Vander Esch).,C,TRADITIONAL,1.14,0,0
2,SHOTGUN,(2:00) (Shotgun) D.Mills pass short right to D.Pierce to HST 26 for 6 yards (D.Walker).,C,TRADITIONAL,3.20,0,0
3,SHOTGUN,(9:28) (Shotgun) P.Mahomes pass short left to I.Pacheco to TEN 19 for 4 yards (Z.Cunningham).,C,TRADITIONAL,3.02,0,0
4,PISTOL,(2:16) (Shotgun) L.Jackson up the middle to TB 28 for -1 yards (R.Nunez-Roches).,NoPass,DESIGNED_RUN,2.03,1,0


In [179]:
plays.loc[0, ['offenseFormation', 'playDescription']]

offenseFormation                                                                                       EMPTY
playDescription     (1:54) (Shotgun) J.Burrow pass short middle to T.Boyd to CIN 30 for 9 yards (J.Hawkins).
Name: 0, dtype: object

`offenseFormation` has 188 missing values из них 165 с kneels - true и 23 - с kneels false.

In [180]:
plays.query("offenseFormation.isna()")['playDescription']

7            (:35) K.Murray kneels to CAR 29 for -1 yards.
25            (:10) G.Smith kneels to SEA 24 for -1 yards.
53         (1:16) J.Brissett kneels to CIN 10 for no gain.
173           (1:31) C.Rush kneels to DAL 38 for -1 yards.
180          (:05) R.Wilson kneels to DEN 24 for -1 yards.
                               ...                        
13671       (:11) J.Herbert kneels to LAC 34 for -1 yards.
13727        (:06) A.Rodgers kneels to GB 15 for -1 yards.
13990    (:18) T.Tagovailoa kneels to MIA 19 for -1 yards.
14556          (:39) C.Rush kneels to NYG 31 for -1 yards.
14565        (:13) R.Wilson kneels to HST 25 for -1 yards.
Name: playDescription, Length: 188, dtype: object

In [181]:
plays.qbKneel.sum()

164

# Coverage

###  `pff_manZone`, `pff_passCoverage` ✅

In [182]:
plays.pff_manZone.unique()

array(['Zone', 'Man', nan, 'Other'], dtype=object)

In [183]:
plays.pff_passCoverage.unique()

array(['Cover-3', 'Quarters', 'Cover-1', 'Cover 6-Left', 'Cover-2', nan,
       'Cover-0', 'Cover-3 Seam', 'Cover-6 Right', 'Goal Line',
       'Red Zone', '2-Man', 'Cover-3 Double Cloud', 'Bracket',
       'Miscellaneous', 'Cover-3 Cloud Left', 'Prevent', 'Cover-1 Double',
       'Cover-3 Cloud Right'], dtype=object)

In [184]:
plays.query('pff_manZone.isna()')['pff_passCoverage'].unique()

array([nan], dtype=object)

In [185]:
plays.query('pff_passCoverage.isna()')['pff_manZone'].unique()

array([nan], dtype=object)

In [186]:
plays.query('pff_manZone=="Other"')['pff_passCoverage'].unique()

array(['Goal Line', 'Red Zone', 'Bracket', 'Miscellaneous', 'Prevent'],
      dtype=object)

In [187]:
import pandas as pd 

In [188]:
players =  pd.read_csv('nfl_data/players.csv')
pp =  pd.read_csv('nfl_data/player_play.csv')
pp = pp.merge(plays, on=['gameId', 'playId']).merge(players[['position', 'nflId']], on='nflId', how='left')

In [189]:
display(pp.query('pff_manZone=="Man" and pff_defensiveCoverageAssignment.isna()')[['pff_manZone','pff_defensiveCoverageAssignment']].head())
display(pp.query('pff_manZone=="Zone" and pff_defensiveCoverageAssignment.isna()')[['pff_manZone','pff_defensiveCoverageAssignment']].head())

,pff_manZone,pff_defensiveCoverageAssignment
132,Man,NaN
133,Man,NaN
134,Man,NaN
135,Man,NaN
136,Man,NaN


,pff_manZone,pff_defensiveCoverageAssignment
0,Zone,NaN
1,Zone,NaN
2,Zone,NaN
3,Zone,NaN
4,Zone,NaN


In [190]:
pp['pff_defensiveCoverageAssignment'] = pp['pff_defensiveCoverageAssignment'].where(pp['pff_defensiveCoverageAssignment'].notna(),
        pp['pff_manZone'].map({'Man': 'MAN', 'Zone': 'ZONE'}))

In [191]:
pp.loc[132][['pff_manZone','pff_defensiveCoverageAssignment']]

pff_manZone                        Man
pff_defensiveCoverageAssignment    MAN
Name: 132, dtype: object

In [192]:
pp.loc[4][['pff_manZone','pff_defensiveCoverageAssignment']]

pff_manZone                        Zone
pff_defensiveCoverageAssignment    ZONE
Name: 4, dtype: object

In [193]:
pp.pff_defensiveCoverageAssignment.unique()

array(['ZONE', 'HCR', '4OR', 'FL', 'CFR', '4IR', '2L', 'HCL', 'CFL', '3L',
       'HOL', '3R', '3M', 'MAN', 'DF', '2R', 'FR', '4OL', '4IL', nan,
       'PRE'], dtype=object)

###  `sackYardsAsOffense`, `sackYardsAsDefense` ✅

In [194]:
pp[['sackYardsAsOffense', 'timeToSack']]

pp.query('timeToSack.isna()')['sackYardsAsOffense'].sum()

0

In [195]:
pp.query('timeToSack.isna()')['sackYardsAsDefense'].sum()

0

### `routeRan`

In [204]:
pp.query('routeRan.isna() and hadPassReception==1')[['passResult', 'position']]

,passResult,position
76453,R,WR
159984,C,QB
215423,C,QB
273136,C,QB
294381,C,QB


In [209]:
pp.query('hadPassReception==1 and position=="QB"')

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,...,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,distanceFromOwnEndZone,penalty,passAfterScramble,position
3075,2022091100,783,45244,NO,0,0,0,0,0,1,...,1,UNKNOWN,UNKNOWN,0,Cover-1,Man,70,0,0,QB
153081,2022100210,867,53431,NYJ,0,0,0,0,0,1,...,1,TRICK,UNKNOWN,0,Red Zone,Other,98,0,0,QB
159984,2022100212,2811,38605,DEN,0,0,1,1,0,1,...,1,UNKNOWN,UNKNOWN,0,Cover-6 Right,Zone,1,0,0,QB
215423,2022101601,3890,54602,NE,0,0,1,-6,0,1,...,1,UNKNOWN,UNKNOWN,0,Cover-3,Zone,48,0,0,QB
246128,2022102000,1104,45244,NO,0,0,0,0,0,1,...,1,UNKNOWN,UNKNOWN,0,Cover-0,Man,97,0,0,QB
273136,2022102309,3333,52414,LAC,0,0,1,-8,0,1,...,1,UNKNOWN,UNKNOWN,0,Cover-3 Seam,Zone,33,0,0,QB
294381,2022103003,2051,43290,DET,0,0,1,5,0,1,...,1,UNKNOWN,UNKNOWN,0,Cover-3,Zone,46,0,0,QB
300889,2022103006,533,45244,NO,0,0,0,0,0,1,...,1,UNKNOWN,UNKNOWN,0,Cover-2,Zone,20,0,0,QB


In [197]:
# pp = pp.drop(['hadDropback'])

In [198]:
# pp = pp.drop(['qbSpike', 'qbKneel', 'isDropback', 
#                     'dropbackDistance', 'timeToSack', 'penalty','penaltyYards'], axis=1)

In [ ]:
# pp = pp.drop(['qbSpike', 'qbKneel', 'isDropback', 'dropbackDistance', 'timeToSack', 'penalty','penaltyYards', 'hadDropback',
#              'penaltyNames', 'timeToPressureAsPassRusher'], axis=1)

# pp.to_csv('plays_new.csv', sep='\t')